# python实现水果多分类问题

### 1.图像处理：
#### 利用Otsu法将图像二值话，实现去噪音的功能
### 2.颜色特征提取：
#### 将RGB的一、二、三阶矩作为特征（9维）；重组HSV形成81维特征
### 3.纹理特征提取：
#### 利用LBP算子得到纹理特征，并计算一、二、三阶矩以及平滑度作为特征（4维）
### 4.构建分类器：
#### 输入85维特征及对应标签，构建3个隐藏层每层15个神经元的分类器
### 5.预测及评分

In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from skimage.feature import local_binary_pattern
from sklearn.neural_network import MLPClassifier

### 读取图片并做预处理：利用Otsu法将图片分割为二值图像

In [2]:
def form_img(path, pic_name):
    pic_path = os.path.join(path,pic_name)
    img = cv2.imread(pic_path)
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    (t,thresh) = cv2.threshold(gray_img,0,255,cv2.THRESH_TOZERO_INV+cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
    thresh = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
    final_pic = cv2.cvtColor(cv2.bitwise_and(img, img,mask=thresh),cv2.COLOR_BGR2RGB)
    return final_pic

### 颜色特征提取：RGB颜色矩阵中的一、二、三阶作为特征

In [3]:
def color_moments(img):
    if img is None:
        return
    # Split the channels - R,G,B
    r = img[:,:,0]
    g = img[:,:,1]
    b = img[:,:,2]
    # Initialize the color feature
    color_feature = []
    # N = h.shape[0] * h.shape[1]
    # The first central moment - average 
    r_mean = np.mean(r)  # np.sum(h)/float(N)
    g_mean = np.mean(g)  # np.sum(s)/float(N)
    b_mean = np.mean(b)  # np.sum(v)/float(N)
    color_feature.extend([r_mean, g_mean, b_mean])
    # The second central moment - standard deviation
    r_std = np.std(r)  # np.sqrt(np.mean(abs(h - h.mean())**2))
    g_std = np.std(g)  # np.sqrt(np.mean(abs(s - s.mean())**2))
    b_std = np.std(b)  # np.sqrt(np.mean(abs(v - v.mean())**2))
    color_feature.extend([r_std, g_std, b_std])
    # The third central moment - the third root of the skewness
    r_skewness = np.mean(abs(r - r.mean())**3)
    g_skewness = np.mean(abs(g - g.mean())**3)
    b_skewness = np.mean(abs(b - b.mean())**3)
    r_thirdMoment = r_skewness**(1./3)
    g_thirdMoment = g_skewness**(1./3)
    b_thirdMoment = b_skewness**(1./3)
    color_feature.extend([r_thirdMoment, g_thirdMoment, b_thirdMoment])

    return color_feature

### 颜色特征提取：HSV非均匀量化

In [4]:
hlist = [20, 40, 75, 155, 190, 270, 290, 316, 360]
svlist = [21, 178, 255]

def quantilize(h, s, v):
    '''hsv直方图量化'''
    # value : [21, 144, 23] h, s, v
    h = h * 2
    for i in range(len(hlist)):
        if h <= hlist[i]:
            h = i % 8
            break
    for i in range(len(svlist)):
        if s <= svlist[i]:
            s = i
            break
    for i in range(len(svlist)):
        if v <= svlist[i]:
            v = i
            break
    return 9 * h + 3 * s + v

In [5]:
def colors(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    quantilize_ufunc = np.frompyfunc(quantilize, 3, 1)
    nhsv = quantilize_ufunc(hsv[:,:,0], hsv[:,:,1], hsv[:,:,2]).astype(np.uint8) # 由于frompyfunc函数返回结果为对象，所以需要转换类型
    hist = cv2.calcHist([nhsv], [0], None, [72], [0,71]) # 40x faster than np.histogram
    hist = hist.reshape(1, hist.shape[0]).astype(np.int32).tolist()[0]
    return hist

### 纹理特征提取：LBP

In [6]:
def hist_param(hist):
    lbp_hist_M = np.mean(hist)
    lbp_hist_Sigma = np.std(hist)
    lbp_hist_Sk = np.mean(abs(hist - hist.mean())**3)
    lbp_hist_Th = lbp_hist_Sk**(1./3) 
    return [lbp_hist_M,lbp_hist_Sigma,lbp_hist_Sk,lbp_hist_Th]

In [7]:
def lbp_trans(img):
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(image_gray, 8, 1, 'nri_uniform')
    max_bins = int(lbp.max() + 1)
    lbp_hist = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))[0]
    return lbp_hist

### 归一化数据

In [8]:
def min_max_scaler(l):
    l_min = np.min(l)
    l_max = np.max(l)
    mm_scaler = [(x-l_min)/(l_max-l_min) for x in l]
    return mm_scaler

### 生成85维特征结果

In [9]:
def form_res(path,pic_name):
    img = form_img(path,pic_name)
    tmp1 = color_moments(img)
    res1 = min_max_scaler(tmp1)
    tmp2 = colors(img)
    res2 = min_max_scaler(tmp2)
    lbp_hist = lbp_trans(img)
    tmp3 = hist_param(lbp_hist)
    res3 = min_max_scaler(tmp3)
    result = res1+res2+res3
    return result

### 构建样本集：训练集(220张图片）与测试集（55张图片）

In [14]:
def train_test_set(set_path):
    list_name = os.listdir(set_path)
    label = [x.split("_")[0] for x in list_name]
    replace_dict={'apple':1,'banana':2,'orange':3}
    y = [replace_dict[i] if i in replace_dict else i for i in label]
    y = np.array(y)
    
    X = np.zeros(85)
    for p in list_name:
        tmp = form_res(set_path,p)
        X = np.vstack((X,tmp))
    X = X[1:]
    return X, y

In [15]:
train_set = r'C:\Users\shallyzhang\Desktop\MZ06\train'
X_train, y_train = train_test_set(train_set)

<ipython-input-7-f6215002c770>:6: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  lbp_hist = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))[0]


In [16]:
test_set = r'C:\Users\shallyzhang\Desktop\MZ06\test'
X_test, y_test = train_test_set(test_set)

<ipython-input-7-f6215002c770>:6: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  lbp_hist = np.histogram(lbp, normed=True, bins=max_bins, range=(0, max_bins))[0]


In [43]:
len(y_train),len(y_test)

(220, 55)

### 构建分类器：隐藏层3层，每层15个神经元，循环5W次

In [40]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(15,15,15),max_iter=50000)
mlp.fit(X_train,y_train)
predictions = mlp.predict(X_test)

### 观测分类结果：1类（苹果）准确率90%，2类（香蕉）94%，3类（桔子）84%

In [41]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

[[18  0  1]
 [ 1 15  2]
 [ 1  1 16]]
              precision    recall  f1-score   support

           1       0.90      0.95      0.92        19
           2       0.94      0.83      0.88        18
           3       0.84      0.89      0.86        18

    accuracy                           0.89        55
   macro avg       0.89      0.89      0.89        55
weighted avg       0.89      0.89      0.89        55

